# HW 3: Neural Machine Translation

In this homework you will build a full neural machine translation system using an attention-based encoder-decoder network to translate from German to English. The encoder-decoder network with attention forms the backbone of many current text generation systems. See [Neural Machine Translation and Sequence-to-sequence Models: A Tutorial](https://arxiv.org/pdf/1703.01619.pdf) for an excellent tutorial that also contains many modern advances.

## Goals


1. Build a non-attentional baseline model (pure seq2seq as in [ref](https://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf)). 
2. Incorporate attention into the baseline model ([ref](https://arxiv.org/abs/1409.0473) but with dot-product attention as in class notes).
3. Implement beam search: review/tutorial [here](http://www.phontron.com/slides/nlp-programming-en-13-search.pdf)
4. Visualize the attention distribution for a few examples. 

Consult the papers provided for hyperparameters, and the course notes for formal definitions.

This will be the most time-consuming assignment in terms of difficulty/training time, so we recommend that you get started early!

## Setup

This notebook provides a working definition of the setup of the problem itself. Feel free to construct your models inline, or use an external setup (preferred) to build your system.

In [2]:
# Text text processing library and methods for pretrained word embeddings
from torchtext import data
from torchtext import datasets
import torch
from torchtext.vocab import Vectors, GloVe
import torch.autograd as autograd
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math
from torch.optim.lr_scheduler import MultiStepLR
from matplotlib import pyplot as plt
from torch.nn.utils import clip_grad_norm
import numpy as np
import torch.nn.init as weight_init
import time
import os
import heapq as hq

We first need to process the raw data using a tokenizer. We are going to be using spacy, which can be installed via:  
  `[sudo] pip install spacy`  
  
Tokenizers for English/German can be installed via:  
  `[sudo] python -m spacy download en`  
  `[sudo] python -m spacy download de`
  
This isn't *strictly* necessary, and you can use your own tokenization rules if you prefer (e.g. a simple `split()` in addition to some rules to acccount for punctuation), but we recommend sticking to the above.

In [3]:
import spacy
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]


Note that we need to add the beginning-of-sentence token `<s>` and the end-of-sentence token `</s>` to the 
target so we know when to begin/end translating. We do not need to do this on the source side.

In [4]:
BOS_WORD = '<s>'
EOS_WORD = '</s>'
DE = data.Field(tokenize=tokenize_de)
EN = data.Field(tokenize=tokenize_en, init_token = BOS_WORD, eos_token = EOS_WORD) # only target needs BOS/EOS

Let's download the data. This may take a few minutes.

**While this dataset of 200K sentence pairs is relatively small compared to others, it will still take some time to train. So we are going to be only working with sentences of length at most 20 for this homework. Please train only on this reduced dataset for this homework.**

In [5]:
MAX_LEN = 20
train, val, test = datasets.IWSLT.splits(exts=('.de', '.en'), fields=(DE, EN), 
                                         filter_pred=lambda x: len(vars(x)['src']) <= MAX_LEN and 
                                         len(vars(x)['trg']) <= MAX_LEN)
print(train.fields)
print(len(train))
print(vars(train[0]))

{'trg': <torchtext.data.field.Field object at 0x7f324dec0ac8>, 'src': <torchtext.data.field.Field object at 0x7f32c03c4eb8>}
119076
{'src': ['David', 'Gallo', ':', 'Das', 'ist', 'Bill', 'Lange', '.', 'Ich', 'bin', 'Dave', 'Gallo', '.'], 'trg': ['David', 'Gallo', ':', 'This', 'is', 'Bill', 'Lange', '.', 'I', "'m", 'Dave', 'Gallo', '.']}


Now we build the vocabulary and convert the text corpus into indices. We are going to be replacing tokens that occurred less than 5 times with `<unk>` tokens, and take the rest as our vocab.

In [6]:
MIN_FREQ = 5
DE.build_vocab(train.src, min_freq=MIN_FREQ)
EN.build_vocab(train.trg, min_freq=MIN_FREQ)
print(DE.vocab.freqs.most_common(10))
print("Size of German vocab", len(DE.vocab))
print(EN.vocab.freqs.most_common(10))
print("Size of English vocab", len(EN.vocab))
print(EN.vocab.stoi["<s>"], EN.vocab.stoi["</s>"]) #vocab index for <s>, </s>

[('.', 113253), (',', 67237), ('ist', 24189), ('die', 23778), ('das', 17102), ('der', 15727), ('und', 15622), ('Sie', 15085), ('es', 13197), ('ich', 12946)]
Size of German vocab 13353
[('.', 113433), (',', 59512), ('the', 46029), ('to', 29177), ('a', 27548), ('of', 26794), ('I', 24887), ('is', 21775), ("'s", 20630), ('that', 19814)]
Size of English vocab 11560
2 3


Now we split our data into batches as usual. Batching for MT is slightly tricky because source/target will be of different lengths. Fortunately, `torchtext` lets you do this by allowing you to pass in a `sort_key` function. This will minimizing the amount of padding on the source side, but since there is still some padding you will inadvertendly "attend" to these padding tokens. 

One way to get rid of padding is to pass a binary `mask` vector to your attention module so its attention score (before the softmax) is minus infinity for the padding token. Another way (which is how we do it for our projects, e.g. opennmt) is to manually sort data into batches so that each batch has exactly the same source length (this means that some batches will be less than the desired batch size, though).

However, for this homework padding won't matter too much, so it's fine to ignore it.

In [7]:
BATCH_SIZE = 32
train_iter, val_iter = data.BucketIterator.splits((train, val), batch_size=BATCH_SIZE, device=-1,
                                                  repeat=False, sort_key=lambda x: len(x.src))

Let's check to see that the BOS/EOS token is indeed appended to the target (English) sentence.

In [8]:
batch = next(iter(train_iter))
print("Source")
print(batch.src)
print("Target")
print(batch.trg)


Source
Variable containing:

Columns 0 to 12 
  104    39   411    28     9    17   133    26    23    39   341   517   298
   33     0  1678   240   111  3435  7090   192  7071     0     6    24     2
   13    54     6    49    24     3   602    41  1693  1678    18  8660    73
   22   434    16    18    17  1702    10   891     3    44  1073   140     3
  641   177  6644    65  3788   704   130    53   224  4032    16   167   106
  484   692    24    51   130   140  1451    37    21     3    23  2307   107
   14  3260    17  1505    15    41     3     5    41   816     4    83  1447
   71     8   204     7  1743     0   898  2832  4801     5    98   446     7
    0  9820     2  2798  1637    16    11     0     4     0   850  2938  7239
    2     2    17     2     2    17     2     2     2  3515     2     2     2

Columns 13 to 25 
   28    79    28    73  1844    77  1002    28     0   180    20  2675   689
   33     9   217    85     4    89     0    33    13     0    67   151     9

Success! Now that we've processed the data, we are ready to begin modeling.

## Assignment

Now it is your turn to build the models described at the top of the assignment. 

When a model is trained, use the following test function to produce predictions, and then upload to the kaggle competition: https://www.kaggle.com/c/cs287-hw3-s18/

For the final Kaggle test, we will provide the source sentence, and you are to predict the **first three words of the target sentence**. The source sentence can be found under `source_test.txt`

In [9]:
!head source_test.txt

Als ich in meinen 20ern war , hatte ich meine erste Psychotherapie-Patientin .
Ich war Doktorandin und studierte Klinische Psychologie in Berkeley .
Sie war eine 26-jährige Frau namens Alex .
Und als ich das hörte , war ich erleichtert .
Meine Kommilitonin bekam nämlich einen Brandstifter als ersten Patienten .
Und ich bekam eine Frau in den 20ern , die über Jungs reden wollte .
Das kriege ich hin , dachte ich mir .
Aber ich habe es nicht hingekriegt .
Arbeit kam später , Heiraten kam später , Kinder kamen später , selbst der Tod kam später .
Leute in den 20ern wie Alex und ich hatten nichts als Zeit .


Similar to HW1, you are to predict the 100 most probable 3-gram that will begin the target sentence. The submission format will be as follows, where each word in the 3-gram will be separated by "|", and each 3-gram will be separated by space. For example, here is what an example submission might look like with 5 most-likely 3-grams (instead of 100).

```
id,word
1,Newspapers|talk|about When|I|was Researchers|call|the Twentysomethings|like|Alex But|before|long
2,That|'s|what Newspapers|talk|about You|have|robbed It|'s|realizing My|parents|wanted
3,We|forget|how We|think|about Proust|actually|links Does|any|other This|is|something
4,But|what|do And|it|'s They|'re|on My|name|is It|only|happens
```

When you print out your data, you will need to escape quotes and commas with the following command so that Kaggle does not complain. 

In [10]:
use_cuda = torch.cuda.is_available()

In [11]:
class Seq2SeqAttn(nn.Module):
    def __init__(self, encoder, decoder, use_true = True):
        super(Seq2SeqAttn, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.last_hidden_enc = (self.encoder.num_layers*(self.encoder.bidirectional*2)) - 1
        self.use_true = use_true # want to feed true last words in when training
        
    def forward(self, input_seqs, target_seqs):
        batch_size = input_seqs.size(1)
        target_length = target_seqs.size(0)
        vocab_size = self.decoder.output_size
        
        outputs = Variable(torch.zeros(target_length, batch_size, vocab_size)).cuda()
        
        encoder_output, hidden = self.encoder(input_seqs, None)
        
        output = Variable(target_seqs.data[0, :])
        
        for t in range(1, target_length):
            
            
            context = Variable(torch.zeros(batch_size,self.encoder.hidden_size)).cuda()
            
            #print(context.size())
            
            #loop over examples in batch
            for i in range(batch_size):
                
                #print(encoder_output[:,i,:].size())
                #print(hidden[0][1,i,:].size())
                
                #calculate attention probabilities
                att_probs = torch.matmul(encoder_output[:,i,:],hidden[0][self.last_hidden_enc,i,:])
                
                #print(att_probs.size())
                
                #now get "expected" for each element in batch
                context[i,:] = torch.matmul(att_probs,encoder_output[:,i,:])
            
            
            output, hidden = self.decoder(output, hidden, context)
            
            outputs[t] = output
            
            #use true values only if training/validating model
            if self.use_true:
                output = Variable(target_seqs.data[t]).cuda()

        return outputs
    
    def batch_train(self, optimizer, train_iter, vocab_size, grad_clip=10):
        self.train()
        total_loss = 0
        pad = EN.vocab.stoi['<pad>']
        curr_time = time.time()
        loss_f = nn.NLLLoss()
        for b, batch in enumerate(train_iter):
            source = batch.src
            target = batch.trg
            if use_cuda:
                source, target = source.cuda(), target.cuda()
            optimizer.zero_grad()
            output = self.forward(source, target)
            loss = F.cross_entropy(output[1:].view(-1, vocab_size),
                                   target[1:].contiguous().view(-1),
                                   ignore_index=pad)
            loss.backward()
            clip_grad_norm(self.parameters(), grad_clip)
            optimizer.step()
            total_loss += loss.data[0]

            if b % 1000 == 0 and b != 0:
                total_loss = total_loss / 1000
                print("[%d][loss:%5.2f][pp:%5.2f][time:%5.2f]" %
                      (b, total_loss, math.exp(total_loss), time.time() - curr_time))
                total_loss = 0
                curr_time = time.time()
    
    def beam_search(self, input_seq, beam_size, search_time, target_vocab, n = 10):
        
        encoder_output, hidden = self.encoder(input_seq)
        
        #here we track the strings still in our beam
        track = [None]*beam_size
        
        # in worst case, take beam_size candidates from one former state
        poss_next = np.min([beam_size, len(target_vocab)])
        
        # next states that we might want to keep around
        possible_next = [None]*(beam_size*poss_next)
        
        # list of complete strings to keep around
        final_candidates = []
        
        # first state we have in our beam
        track[0] = ([target_vocab.stoi["<s>"]], hidden, 0)
        
        #start time
        start = time.time()
        
        #track no. of steps we've taken
        steps = 0
        
        #continue looping until we've used search time
        while time.time() < search_time:
            
            steps += 1
            
            #track where we are in the list of possible next values
            poss_counter = 0
            
            for i in range(len(track)):
                
                att_probs = torch.matmul(encoder_output[:,0,:],track[i][1][0][self.last_hidden_enc,0,:])
            
                context = torch.matmul(att_probs,encoder_output[:,0,:])
                
                output, hidden = self.decoder(torch.Tensor(track[i][-1]), hidden, context)
                
                log_output = torch.log(output)
                
                top_next, idx = torch.topk(log_output, poss_next)
                
                for j in range(len(top_next)):
                    
                    if idx[j] != target_vocab.stoi["</s>"]:
                        
                        possible_next[poss_counter+j] = (log_output[idx[j]] + track[i][2],top_next[j], idx[j], i, steps)
                    
                        poss_counter += 1
                        
                    else:
                        
                        #we have a complete string
                        final_candidates += [(top_next[j] + track[i][2],track[i][0]+[idx[j]])]
                    
                    
            if poss_counter < beam_search:
                
                print("less possible next states than beam size. Seems unlikely this would ever happen.")
                
                break
                
            #sort our possible next states and choose
            hq.heapify(possible_next[:poss_counter])
            largest = hq.nlargest(beam_size, possible_next[:poss_counter])
            
            #now loop over and put everything we want to keep in our track array
            for k in range(poss_counter):
                parent = track[possible_next[k][i]]
                track[k] = (parent[0] + [possible_next[k][1]], hidden, possible_next[k][0])
                
        #finally, sort our final candidates
        hq.heapify(final_candidates)
        final = hq.nlargest(n, final_candidates)
        
        return final
                
            
        
    def predict(self, val_iter, vocab_size):
        self.eval()
        pad = EN.vocab.stoi['<pad>']
        total_loss = 0
        for batch in val_iter:
            source = batch.src
            target = batch.trg
            if use_cuda:
                source = Variable(source.data.cuda(), volatile=True)
                target = Variable(target.data.cuda(), volatile=True)
            output = self.forward(source, target)
            loss = F.cross_entropy(output[1:].view(-1, vocab_size),
                                   target[1:].contiguous().view(-1),
                                   ignore_index=pad)
            total_loss += loss.data[0]
        return total_loss / len(val_iter)

In [13]:
class AttnDecoderLSTM(nn.Module):
    def __init__(self, hidden_size=200, context_size=200, output_size=11560, n_layers = 2, dropout= 0.3, bidirectional=None):
        super(AttnDecoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.context_size = context_size
        self.bidirectional = bidirectional
        self.dropout = nn.Dropout(dropout, inplace=True)
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(self.hidden_size+self.context_size, self.hidden_size, self.n_layers, bidirectional=bidirectional)
        self.out = nn.Linear(self.hidden_size+self.context_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input_seq, hidden, context):
        embedded = self.embedding(input_seq).unsqueeze(0)
        embedded = self.dropout(embedded)
        combined = torch.cat([embedded, context.unsqueeze(0)],2)
        output, hidden = self.lstm(combined, hidden)
        output = F.tanh(output)
        output = self.softmax(self.out(torch.cat([output,context.unsqueeze(0)],2)))
        return output, hidden

In [14]:
class EncoderLSTM(nn.Module):
    def __init__(self, input_size = 13352, hidden_size = 200, n_layers=2, dropout=0.3, bidirectional=None):
        super(EncoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, n_layers, bidirectional=bidirectional)

    def forward(self, input, hidden):
        embedded = self.embedding(input)
        output, hidden = self.lstm(embedded, hidden)
        return output, hidden
        
class DecoderLSTM(nn.Module):
    def __init__(self, hidden_size=200, output_size=11560, n_layers = 2, dropout= 0.3, bidirectional=None):
        super(DecoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = nn.Dropout(dropout, inplace=True)
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(self.hidden_size, self.hidden_size, self.n_layers, bidirectional=bidirectional)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input_seq, hidden):
        embedded = self.embedding(input_seq).unsqueeze(0)
        embedded = self.dropout(embedded)
        output, hidden = self.lstm(embedded, hidden)
        output = F.tanh(output)
        output = self.softmax(self.out(output))
        return output, hidden

In [15]:
encoder = EncoderLSTM(input_size = len(DE.vocab), hidden_size = 200, n_layers=2, dropout=0.3, bidirectional=None)
decoder = AttnDecoderLSTM(hidden_size=200, context_size = 200, output_size = len(EN.vocab), n_layers = 2, dropout= 0.3, bidirectional=None)
seq2seq = Seq2SeqAttn(encoder, decoder).cuda()
epoch_num = 13
optimizer = optim.SGD(seq2seq.parameters(), lr=1)
scheduler = MultiStepLR(optimizer, milestones=range(9, epoch_num), gamma=0.5)

In [16]:
best_val_loss = None
for i in range(epoch_num):
    seq2seq.batch_train(optimizer, train_iter,len(EN.vocab), grad_clip = 10)
    val_loss = seq2seq.predict(val_iter, len(EN.vocab))
    print("[Epoch:%d] val_loss:%5.3f | val_pp:%5.2fS"
          % (i, val_loss, math.exp(val_loss)))

    # Save the model if the validation loss is the best we've seen so far.
    if not best_val_loss or val_loss < best_val_loss:
        print("[!] saving model...")
        if not os.path.isdir(".save"):
            os.makedirs(".save")
        torch.save(seq2seq.state_dict(), './.save/seq2seq_%d.pt' % (i))
        best_val_loss = val_loss
test_loss = seq2seq.predict(test_iter, len(EN.vocab))
print("[TEST] loss:%5.2f" % test_loss)

KeyboardInterrupt: 

In [147]:
def escape(l):
    return l.replace("\"", "<quote>").replace(",", "<comma>")

In [ ]:
best_val_loss = None
seq2seq.load_state_dict(torch.load("./.save/seq2seq_0.pt"))
for i in range(epoch_num):
    seq2seq.batch_train(optimizer, train_iter,len(EN.vocab), grad_clip = 10)
    val_loss = seq2seq.predict(val_iter, len(EN.vocab))
    print("[Epoch:%d] val_loss:%5.3f | val_pp:%5.2fS"
          % (i, val_loss, math.exp(val_loss)))

    # Save the model if the validation loss is the best we've seen so far.
    if not best_val_loss or val_loss < best_val_loss:
        print("[!] saving model...")
        if not os.path.isdir(".save"):
            os.makedirs(".save")
        torch.save(seq2seq.state_dict(), './.save/seq2seq_%d.pt' % (i))
        best_val_loss = val_loss
test_loss = seq2seq.predict(test_iter, len(EN.vocab))
print("[TEST] loss:%5.2f" % test_loss)

[1000][loss: 5.18][pp:177.07][time:661.47]
[2000][loss: 4.93][pp:138.02][time:656.48]
[3000][loss: 4.78][pp:119.11][time:652.24]
[Epoch:0] val_loss:4.500 | val_pp:90.04S
[!] saving model...
[1000][loss: 4.51][pp:90.75][time:648.96]
[2000][loss: 4.40][pp:81.60][time:653.10]
[3000][loss: 4.32][pp:75.41][time:648.10]
[Epoch:1] val_loss:4.173 | val_pp:64.90S
[!] saving model...
[1000][loss: 4.20][pp:66.53][time:649.66]
[2000][loss: 4.10][pp:60.32][time:644.87]
[3000][loss: 4.06][pp:58.19][time:649.94]
[Epoch:2] val_loss:4.241 | val_pp:69.49S
[1000][loss: 4.03][pp:56.35][time:646.50]
[2000][loss: 3.98][pp:53.44][time:648.41]
[3000][loss: 3.91][pp:49.93][time:649.93]
[Epoch:3] val_loss:3.862 | val_pp:47.58S
[!] saving model...
[1000][loss: 3.79][pp:44.34][time:648.55]
[2000][loss: 3.80][pp:44.86][time:647.78]
[3000][loss: 3.74][pp:42.06][time:650.32]
[Epoch:4] val_loss:3.662 | val_pp:38.96S
[!] saving model...
[1000][loss: 3.65][pp:38.66][time:649.32]
[2000][loss: 3.68][pp:39.72][time:647.94

You should perform your hyperparameter search/early stopping/write-up based on perplexity, not the above metric. (In practice, people use a metric called [BLEU](https://www.aclweb.org/anthology/P02-1040.pdf), which is roughly a geometric average of 1-gram, 2-gram, 3-gram, 4-gram precision, with a brevity penalty for producing translations that are too short.)

Finally, as always please put up a (short) write-up following the template provided in the repository:  https://github.com/harvard-ml-courses/cs287-s18/blob/master/template/
